In [1]:
import ROOT
from ROOT import TFile
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import copy
import pandas as pd 
import time
import root_pandas as rpd
from root_pandas import read_root

mpl.rcParams.update({'font.size': 18})
mpl.rcParams.update({'legend.fontsize': 20})
mpl.rcParams.update({'xtick.labelsize': 18}) 
mpl.rcParams.update({'ytick.labelsize': 18}) 
mpl.rcParams.update({'text.usetex' : False})

colors = {}
colors['C'] = '#1f77b4'
colors['Fe'] = '#ff7f0e'
colors['Pb']=  '#2ca02c'

Welcome to JupyROOT 6.22/06


# Function definition to extract the ratios

In [ ]:
def getRatio(df_A,df_D,df_trigger_A,df_trigger_D, variable='h2_z',trig_cut = 'h1_z>0.5', pair_cut='',minz=0.05,
             maxz=0.5,nbins=9, applyweight=False):
    print ('Print Trigger Cut ' ,trig_cut)
    print ('Total Cut ', trig_cut + pair_cut)
    norm_A = df_trigger_A.query(trig_cut).shape[0] 
    norm_D = df_trigger_D.query(trig_cut).shape[0]
    bins= np.linspace(minz,maxz,nbins)
    y_A, x_conditional = np.histogram(df_A.query(trig_cut+pair_cut)[variable],bins=bins)
    y_D, x_conditional = np.histogram(df_D.query(trig_cut+pair_cut)[variable],bins=bins)
    erry_A, x_conditional = np.histogram(df_A.query(trig_cut+pair_cut)[variable],bins=bins)
    erry_D, x_conditional = np.histogram(df_D.query(trig_cut+pair_cut)[variable],bins=bins)
    x_conditional = (x_conditional[1:] + x_conditional[:-1])/2.0
    err_A = np.true_divide(np.sqrt(erry_A),y_A)
    err_D = np.true_divide(np.sqrt(erry_D),y_D)
    y_A = np.true_divide(y_A,norm_A)
    y_D = np.true_divide(y_D,norm_D)
    ratio_conditional = np.true_divide(y_A,y_D)
    error_conditional = np.multiply(ratio_conditional, np.sqrt(np.power(err_A,2.0) + np.power(err_D,2.0)))
    return ratio_conditional,error_conditional,x_conditional



# Definition of functions related to the cuts applied

In [ ]:
def applyCut(inputDataframe, cut, text=None):
    nbeforecut = inputDataframe.shape[0]
    cutDataframe = None
    if nbeforecut>0:
        cutDataframe = inputDataframe.query(cut)
        if text:
            print (text, cutDataframe.shape[0], ' (%2.2f '%(100.0*cutDataframe.shape[0]/nbeforecut), '%)')
    return cutDataframe

def applyCuts(fullDataframe,name='default',isMC=False,isTrigger=True): 
    dataframe = fullDataframe
    if(dataframe.shape[0]>0):
        print ('Entries before cut ', dataframe.shape[0])
    dataframe.eval('inelasticity = nu/5.014', inplace=True)
    dataframe.eval('h1_e = h1_z*nu', inplace=True)
    dataframe.eval('h1_p = sqrt(h1_e*h1_e-0.138*0.138)', inplace=True)
    dataframe = applyCut(dataframe, 'Q2>1.0', 'Q2>1.0 :')
    #dataframe = applyCut(dataframe, 'Nu>3.0 and Nu<3.5', '3.0 < Nu < 3.5')
    #dataframe = applyCut(dataframe, 'h_z>0.5', 'h_z>0.5 :')
    dataframe = applyCut(dataframe, 'h1_p <2.5 ', 'h1_p<2.5 ')
    dataframe = applyCut(dataframe, 'inelasticity<0.85','inelasticity < 0.85')
    return dataframe

def applyCutsPair(fullDataframe,name='default',isMC=False):
    print ('Starting election on dipion variables')
    if (isMC):
        print ('This is MC')
    else: 
        print ('This is Data')
    
    dataframe = fullDataframe
    dataframe.eval('z_tot = h1_z+ h2_z', inplace=True)
    dataframe.eval('h1_e = h1_z*nu', inplace=True)
    dataframe.eval('h1_p = sqrt(h1_e*h1_e-0.138*0.138)', inplace=True)
    dataframe.eval('h2_e = h2_z*nu', inplace=True)
    dataframe.eval('h2_p = sqrt(h2_e*h2_e-0.138*0.138)', inplace=True)
    dataframe = applyCut(dataframe, 'Q2>1.0', 'Q2>1.0 :')
    #dataframe = applyCut(dataframe, 'nu>3.0 and nu<3.5', '3.0 < nu < 3.5')
    dataframe = applyCut(dataframe, 'h1_z>0.5', 'h1_z>0.5 :')
    dataframe = applyCut(dataframe, 'h1_p <2.5 ', 'h1_p<2.5 ')
    #dataframe = applyCut(dataframe, 'h1_pid*h2_pid<0', 'Opposite sign pairs')
    dataframe.eval('pair_pt2 = pair_pt*pair_pt', inplace=True)
    dataframe = applyCut(dataframe, 'h2_pid==2212 | abs(h2_pid)==211', 'h2_pid =2212 or pions') 
    dataframe = applyCut(dataframe, 'h2_p<2.5', 'h2_p <2.5 GeV')

    if (not isMC):
        dataframe = applyCut(dataframe, 'h2_th<120 and h2_th>10', '10< h2_th<120')
        dataframe = applyCut(dataframe, 'h2_p>0.200', 'h2_p>0.200')
        dataframe = applyCut(dataframe, 'h2_pid==2212 | (h2_pid==211)| (h2_pid==-211 & h2_th>30)| (h2_pid==-211 & h2_th<30 & h2_p>0.600)','P>500 for pi- with theta<30')
    
    return dataframe

def printPairBreakdown(dataframe):
    allpairs = 1.0*dataframe.shape[0]
    print ('All pairs ', allpairs)
    print ('Pairs with Leading pi+', np.true_divide(dataframe.query('h1_pid==211').shape[0],allpairs))
    print ('Pairs with Leading pi-', dataframe.query('h1_pid==-211').shape[0]/allpairs)
    print ('Pairs with Sub-Leading pi+', dataframe.query('h2_pid==211').shape[0]/allpairs)
    print ('Pairs with Sub-Leading pi-', dataframe.query('h2_pid==-211').shape[0]/allpairs)
    print ('pi+ pi+ pairs',dataframe.query('h1_pid==211 and h2_pid==211').shape[0]/allpairs)
    print ('pi- pi- pairs',dataframe.query('h1_pid==-211 and h2_pid==-211').shape[0]/allpairs)
    print ('pi+ pi- pairs',dataframe.query('h1_pid==211 and h2_pid==-211').shape[0]/allpairs)
    print ('pi- pi+ pairs',dataframe.query('h1_pid==-211 and h2_pid==211').shape[0]/allpairs)
    print ('//////////////////////////////////////////////////////')
    return


# Get data from Ntuples to dataframes

In [ ]:
#this needs to be read in chunks otherwise it uses all memory. (from https://github.com/scikit-hep/root_pandas)
def getdatainChunks(filename,treename):
    dataframe =pd.DataFrame()
    for df in read_root(filename, treename, chunksize=100000) :
        #print df.shape[0]
        dataframe = pd.concat([dataframe,df], ignore_index=True) #concat is for join two data frames, nothing else
    print (dataframe.shape[0])
    return dataframe

# Getting Data

In [ ]:
df = {}
df_trigger = {}

for target in ['C','Fe','Pb']:
    print ('Getting dataframes from target %s'%target)
    df[target]=getdatainChunks('/home/seba/CreatePairTuples/original_Pairs_%s.root'%target, target)
    df['D_%s'%target]= getdatainChunks('/home/seba/CreatePairTuples/original_Pairs_%s.root'%target, 'D_%s'%target)  
    print ('Get trigger dataframes from deuterium target %s'%target)
    df_trigger['%s_trigger'%target] = getdatainChunks('/home/seba/CreatePairTuples/original_Pairs_%s.root'%target, '%s_trigger'%target)
    df_trigger['D_%s_trigger'%target] = getdatainChunks('/home/seba/CreatePairTuples/original_Pairs_%s.root'%target, 'D_%s_trigger'%target)

# Combine the D2 data-sets

In [ ]:
#df['D'] = df['D_Fe']
#df_trigger['D_trigger'] = df_trigger['D_Fe_trigger']
 #df['D'] = pd.concat([df['D_Fe']) #simply combine the deuterium results for all targets
#df_trigger['D_trigger'] = pd.concat([df_trigger['D_Fe_trigger']]) #simply combine the deuterium results for all targets

#print (df.keys())
#print(df['Fe'].keys())
#print(df_trigger['Fe_trigger'].keys())
## this is just to notice that there are no vectors in the variables, all of them are numbers
#print(df['Fe'].describe())
#print('there are ',len(df['Fe']), 'entries')

watching some distributions

In [ ]:
#df['Fe'].hist(figsize=(12,12),bins=100,column=['dphi','dphi_lab','h2_cm_th','h2_pid'])
#plt.show()

# Apply selection to datasets

In [ ]:
#dataframees with pairs
for key in df.keys():
    isMC = False
    if 'GiBUU' in key:
        isMC=True
    
    print ('\nkey: ',key)
    df[key] = applyCuts(df[key],isMC=isMC)
    df[key] = applyCutsPair(df[key],isMC=isMC)


# R_2h data from HERMES and Neutrino buble-chamber experiment

In [ ]:
#hermes data:
hermes_y = {}
hermes_ystat = {}
hermes_ysyst = {}
hermes_ytotal = {}

#DATA FROM SELECTIO1 OF http://www-hermes.desy.de/notes/pub/publications.html
hermes_x = {}
hermes_x['Kr'] = [0.09,0.15,0.24,0.35,0.44]
hermes_x['N'] = np.subtract(hermes_x['Kr'],0.004)
hermes_x['Xe'] = np.subtract(hermes_x['Kr'],-0.004)

#nitrogen, atomic number 7
hermes_y['N']     = [1.0324, 0.9781,0.9293,0.8678,0.8822]
hermes_ystat['N'] = [0.0767,0.0268,0.0323,0.0478,0.0601]
hermes_ysyst['N'] = [0.0206,0.0196,0.0186,0.0174,0.0176]
hermes_ytotal['N'] = np.sqrt(np.power(hermes_ystat['N'],2.0)+np.power(hermes_ysyst['N'],2.0))

#krypton, atomic number 36
hermes_y['Kr'] = [1.2072,0.9180,0.8822,0.8631,1.0314]
hermes_ystat['Kr'] = [0.1342,0.0366,0.0452,0.0661,0.0972]
hermes_ysyst['Kr'] =[0.0241,0.0184,0.0176,0.0173,0.0206]
hermes_ytotal['Kr'] = np.sqrt(np.power(hermes_ystat['Kr'],2.0)+np.power(hermes_ysyst['Kr'],2.0))

hermes_y['Xe'] = [1.1648,0.9345,0.9242,0.8128,0.9373]
hermes_ystat['Xe'] = [0.1205,0.0359,0.0433,0.0614,0.0808]
hermes_ysyst['Xe'] = [0.0233,0.0187,0.0185,0.0163,0.0187]
hermes_ytotal['Xe'] = np.sqrt(np.power(hermes_ystat['Xe'],2.0)+np.power(hermes_ysyst['Xe'],2.0))


neutrino_x = [0.10763636363636367, 0.18618181818181817,0.25163636363636366,0.3345454545454546]
neutrino_y = [1.4535664335664338, 1.0148378893833438,  1.0373426573426574, 0.8784361093452003]
neutrino_yerror =[1.5794405594405596, 1.112739987285442, 1.1422377622377624,1.004310235219326]
neutrino_yerror = np.subtract(neutrino_yerror,neutrino_y)

fig = plt.figure(figsize=(8,6))

plt.errorbar(hermes_x['N'], hermes_y['N'],yerr=hermes_ytotal['N'],label='Hermes, $^{14}N$',fmt='o')
plt.errorbar(hermes_x['Kr'], hermes_y['Kr'],yerr=hermes_ytotal['Kr'],label='Hermes, $^{84}Kr$',fmt='o')
plt.errorbar(hermes_x['Xe'], hermes_y['Xe'],yerr=hermes_ytotal['Xe'],label='Hermes, $^{131}Xe$',fmt='o')
plt.errorbar(neutrino_x, neutrino_y,yerr=neutrino_yerror,label='SKAT, $A_{eff}=21$',fmt='o')
plt.xlabel('$z_{2}$',fontsize=22)
plt.ylabel('$R_{2h}$',fontsize=22)
plt.axhline(y=1.0,color='black',linestyle='--',alpha=0.5)

plt.tight_layout()
plt.legend(frameon=True)
plt.savefig('PreviousData.png')
plt.savefig('PreviousData_R2h.pdf')

plt.show()


In [ ]:
fig = plt.figure(figsize=(8,6))
## getRatio returns:  ratio_conditional,error_conditional,x_conditional

for i,target in enumerate(['C','Fe','Pb']):
    r, err,x = getRatio(df['%s'%target], df['D_%s'%target], 
                        df_trigger['%s_trigger'%target], df_trigger['D_%s_trigger'%target],
                        minz=0.05,pair_cut='and abs(h2_pid)==211 and h2_z>0.07',maxz=0.45)
    plt.errorbar(x+0.01*i,r,yerr=err,label='This work,  %s'%target,fmt='o-',ms=8) ## 'fmt' is for formatting
    #plt.fill_between(np.subtract(x,-i*0.005), r*0.94,r*1.04, color=colors[target],alpha=0.2)
plt.xlabel('$z_{2}$',fontsize=18)
plt.ylabel('$R_{2h}$',fontsize=18)

plt.errorbar(hermes_x['N'], hermes_y['N'],yerr=hermes_ytotal['N'],fmt='o',mfc='white',label='HERMES, $N$',fillstyle=None,ms=8,capsize=5)
plt.errorbar(hermes_x['Kr'], hermes_y['Kr'],yerr=hermes_ytotal['Kr'],fmt='o',mfc='white',label='HERMES, $Kr$',fillstyle=None,ms=8,capsize=5)
plt.errorbar(hermes_x['Xe'], hermes_y['Xe'],yerr=hermes_ytotal['Xe'],fmt='o',mfc='white',label='HERMES, $Xe$',fillstyle=None,ms=8,capsize=5)
#plt.errorbar(neutrino_x, neutrino_y,yerr=neutrino_yerror,fmt='s',mfc='white',label='Neutrino, $A_{eff}=21$',fillstyle=None,ms=8,capsize=5)

#plt.ylim([0.5,1.5])
#plt.xlim([0.00,0.5])
plt.axhline(y=1.0,color='black',linestyle='--')
plt.legend(loc='best',frameon=False,fontsize=15,ncol=2)

plt.tight_layout()
plt.savefig('ComparisonWithHermes_Linear.png')
#plt.xlim([0.05,0.5])  ##pair_cut='and abs(h2_pid)==211',minz=0.01,maxz=1.0,nbins=20,variable='pair_pt'

#plt.xscale('log')
#plt.savefig('ComparisonWithHermes_Log.png')
plt.grid(True, axis='x',color='gainsboro', linestyle='dashed')
plt.grid(True, axis='y',color='gainsboro', linestyle='dashed')

plt.show()



# As a function of Pair_pt
pair_pt is a variable, is the transverse momentum of the dipion wrt the virtual photon (the usual pt but this time of the dipion)

In [ ]:
fig = plt.figure(figsize=(8,6))

for target in ['C','Fe','Pb']:
    r, err,x = getRatio(df['%s'%target], df['D_%s'%target], df_trigger['%s_trigger'%target],
                        df_trigger['D_%s_trigger'%target],
                        pair_cut='and abs(h2_pid)==211',minz=0.01,maxz=1.0,nbins=20,variable='pair_pt')
    plt.errorbar(np.subtract(x,-i*0.005),r,yerr=err,label='This work, %s'%target,ms=8,fmt='o',alpha=0.99,color=colors[target])
    plt.fill_between(np.subtract(x,-i*0.005), r*0.94,r*1.04, color=colors[target],alpha=0.2)
plt.xlabel('Pair $p_{T}$',fontsize=18)
plt.ylabel('$R_{2h}$',fontsize=18)
plt.axhline(y=1.0,color='black',linestyle='--')
plt.legend(loc='best',frameon=False,fontsize=15,ncol=1)
plt.tight_layout()
plt.grid(True, axis='x',color='gainsboro', linestyle='dashed')
plt.grid(True, axis='y',color='gainsboro', linestyle='dashed')
plt.savefig('R2h_Pt.pdf')

# As a function of proton rapidity

In [ ]:
fig = plt.figure(figsize=(8,6))
for target in ['C','Fe','Pb']:
    r, err,x = getRatio(df['%s'%target], df['D_%s'%target], df_trigger['%s_trigger'%target],
                        df_trigger['D_%s_trigger'%target],
                       pair_cut='and abs(h2_pid)==2212',minz=-1.5,maxz=3.0,nbins=20,variable='h2_rap')
    print('target : ', target)                   
    plt.errorbar(np.subtract(x,-i*0.005),r,yerr=err,label='This work, %s'%target,ms=8,fmt='o',alpha=0.99,color=colors[target])
    plt.fill_between(np.subtract(x,-i*0.005), r*0.94,r*1.04, color=colors[target],alpha=0.2)
    
plt.xlabel('proton rapidity',fontsize=18)
plt.ylabel('$R_{2h}$',fontsize=18)
plt.axhline(y=1.0,color='black',linestyle='--')
plt.legend(loc='best',frameon=False,fontsize=15,ncol=1)
plt.tight_layout()
plt.grid(True, axis='x',color='gainsboro', linestyle='dashed')
plt.grid(True, axis='y',color='gainsboro', linestyle='dashed')
plt.savefig('R2h_ProtonRapidity.png')

# With respect to dphi
'dphi' is a variable , is the difference between the PhiPQ of the trigger hadron and the secondary hadron

In [ ]:
fig = plt.figure(figsize=(8,6))

for target in ['C','Fe','Pb']:
    r, err,x = getRatio(df['%s'%target], df['D_%s'%target], df_trigger['%s_trigger'%target],
                        df_trigger['D_%s_trigger'%target],
                        pair_cut='and abs(h2_pid)==211 and h2_z>0.07 and nu>3.0 and nu<3.5',minz=0.0,maxz=3.14,nbins=20,variable='dphi')
                        #pair_cut='and abs(h2_pid)==2212 and h2_z>0.1 and h2_rap>-1.5 and h2_rap<-1.0',minz=0.0,maxz=3.14,nbins=20,variable='dphi')
                       
    plt.errorbar(np.subtract(x,-i*0.005),r,yerr=err,label='This work, %s'%target,ms=8,fmt='o',alpha=0.99,color=colors[target])
    plt.fill_between(np.subtract(x,-i*0.005), r*0.94,r*1.04, color=colors[target],alpha=0.2)
plt.xlabel('$|\Delta\phi|$ [rad]',fontsize=18)
plt.ylabel('$R_{2h}$',fontsize=18)

#plt.xlim([0.06,0.5])
#plt.ylim([0.0,2.0])

plt.axhline(y=1.0,color='black',linestyle='--')
plt.legend(loc='best',frameon=False,fontsize=15,ncol=1)
plt.grid(True, axis='x',color='gainsboro', linestyle='dashed')
plt.grid(True, axis='y',color='gainsboro', linestyle='dashed')
plt.tight_layout()
plt.savefig('R2h_DeltaPhi.pdf')

In [ ]:
plt.hist(df['Fe']['dphi'],bins=100,alpha=0.5,density=True,label='Iron') # it's normalized due to the density=True!
plt.hist(df['D_Fe']['dphi'],bins=100,alpha=0.5,density=True,label='Deuterium')
plt.xlabel('dphi')
plt.ylabel('Normalized Entries')
plt.legend()
plt.show()

# MC simulations

here are test files:  /home/seba/CreatePairTuples/MC_Pairs_C_test.root
                      /home/seba/CreatePairTuples/MC_Pairs_D_test.root

getting MC data frames

In [ ]:
df_mc = {}
df_mc_trigger = {}

for target in ['C','D', 'Fe','Pb']:
    print ('Getting dataframes from target %s'%target)
    df_mc[target]=getdatainChunks('/home/seba/CreatePairTuples/MC_Pairs_%s.root'%target, target)
    df_mc_trigger['%s_trigger'%target] = getdatainChunks('/home/seba/CreatePairTuples/MC_Pairs_%s.root'%target, '%s_trigger'%target)

In [ ]:
for key in df_mc.keys():
    isMC = False
    if 'GiBUU' in key:
        isMC=True
    
    print (key)
    
    df_mc[key] = applyCuts(df_mc[key],isMC=isMC)
    df_mc[key] = applyCutsPair(df_mc[key],isMC=isMC)    
    
    df_mc[key] = applyCuts(df_mc[key],isMC=isMC)
    df_mc[key] = applyCutsPair(df_mc[key],isMC=isMC)

In [ ]:
for key in df_mc_trigger.keys():
    print (key)
    df_mc_trigger[key] = applyCuts(df_mc_trigger[key],isTrigger=True)
    print (' ')

In [ ]:
fig = plt.figure(figsize=(8,6))

for i,target in enumerate(['Pb']):
    r, err,x = getRatio(df_mc['%s'%target], df_mc['D'], 
                        df_mc_trigger['%s_trigger'%target], df_mc_trigger['D_trigger'],
                        minz=0.08)
    
    plt.errorbar(x+0.01,r,yerr=err,label='all pairs,  %s'%target,fmt='o',ms=8)
    r, err,x = getRatio(df_mc['%s'%target], df_mc['D'], 
                        df_mc_trigger['%s_trigger'%target], df_mc_trigger['D_trigger'],
                        minz=0.08,    pair_cut='and h1_pid>0 and h2_pid<0')
    plt.errorbar(x+0.02,r,yerr=err,label='$\pi^{+}$, $\pi^{-}$',fmt='o',ms=8)
    
    
    r, err,x = getRatio(df_mc['%s'%target], df_mc['D'], 
                        df_mc_trigger['%s_trigger'%target], df_mc_trigger['D_trigger'],
                        minz=0.08,    pair_cut='and h1_pid<0 and h2_pid>0')
    plt.errorbar(x+0.03,r,yerr=err,label='$\pi^{-}$, $\pi^{+}$',fmt='o',ms=8)

plt.title('Simulation (cross-check)')    
plt.ylabel(r'$R_{2h}$',fontsize=22)
plt.xlabel(r'$z_{2}$',fontsize=22)
plt.ylim([0.5,1.5])
plt.grid(True, axis='x',color='gainsboro', linestyle='dashed')
plt.grid(True, axis='y',color='gainsboro', linestyle='dashed')
plt.legend(frameon=True,loc='best')
plt.savefig('pb_mc_check.pdf')


# GIBUU

In [ ]:
pathGIBUU="/home/miguel/eg2dihadron/"
for target in ['D','C','Fe','Pb']:
    #GiBUU
    print (target)
    print (' trigger')
    df_trigger['GiBUU_%s_trigger'%target]= getdatainChunks('%sGiBUU_Pairs_%s.root'%(pathGIBUU,target), '%s_trigger'%target)
    print (' pairs')
    df['GiBUU_%s'%target]= getdatainChunks('%sGiBUU_Pairs_%s.root'%(pathGIBUU,target), target)
print (df.keys())

In [ ]:
#df['GiBUU_C'].query('h2_pid==2212').hist(figsize=(24,24),bins=100)
#plt.yscale('log')
#plt.show()

In [ ]:
for key in df.keys():
    isMC = False
    if 'GiBUU' in key:
        isMC=True
    
    print (key)
    
    df[key] = applyCuts(df[key],isMC=isMC)
    df[key] = applyCutsPair(df[key],isMC=isMC)

In [ ]:
for key in df_mc.keys():
    isMC = False
    if 'GiBUU' in key:
        isMC=True
    
    print (key)
    
    df_mc[key] = applyCuts(df_mc[key],isMC=isMC)
    df_mc[key] = applyCutsPair(df_mc[key],isMC=isMC)    
    
    df_mc[key] = applyCuts(df_mc[key],isMC=isMC)
    df_mc[key] = applyCutsPair(df_mc[key],isMC=isMC)

In [ ]:
df['GiBUU_C'].keys()

In [ ]:
fig = plt.figure(figsize=(8,6))

for target in ['C','Fe','Pb']:
    r, err,x = getRatio(df['GiBUU_%s'%target],df['GiBUU_D'],
                        df_trigger['GiBUU_%s_trigger'%target],df_trigger['GiBUU_D_trigger'],
                        pair_cut='and h2_pid==2212',minz=0.01,maxz=1.0,nbins=20,variable='pair_pt')
    plt.errorbar(x,r,yerr=err,label='GiBUU %s'%target,ms=8,lw=2)
for target in ['C','Fe','Pb']:
    r, err,x = getRatio(df['%s'%target], df['D_%s'%target], df_trigger['%s_trigger'%target],
                        df_trigger['D_%s_trigger'%target],
                        pair_cut='and h2_pid==2212',minz=0.01,maxz=1.0,nbins=20,variable='pair_pt')
                       
    plt.errorbar(np.subtract(x,-i*0.005),r,yerr=err,label='This work, %s'%target,ms=8,fmt='o',alpha=0.99,color=colors[target])
    plt.fill_between(np.subtract(x,-i*0.005), r*0.94,r*1.04, color=colors[target],alpha=0.2)
plt.xlabel('Pair $p_{T}$',fontsize=18)
plt.ylabel('$R_{2h}$',fontsize=18)

#plt.xlim([0.06,0.5])
#plt.ylim([0.0,2.0])

plt.axhline(y=1.0,color='black',linestyle='--')
plt.legend(loc='best',frameon=False,fontsize=15,ncol=1)

plt.grid(True, axis='x',color='gainsboro', linestyle='dashed')
plt.grid(True, axis='y',color='gainsboro', linestyle='dashed')
#plt.xscale('log')

plt.tight_layout()
plt.savefig('TheoryOnly.png')

In [ ]:
fig = plt.figure(figsize=(8,6))

for target in ['C','Fe','Pb']:
    r, err,x = getRatio(df['GiBUU_%s'%target],df['GiBUU_D'],
                        df_trigger['GiBUU_%s_trigger'%target],df_trigger['GiBUU_D_trigger'],
                        pair_cut='and abs(h2_pid)==2212',minz=-1.5,maxz=3.0,nbins=20,variable='h2_rap')
    plt.errorbar(x,r,yerr=err,label='GiBUU %s'%target,ms=8,lw=2)

    
for target in ['C','Fe','Pb']:
    r, err,x = getRatio(df['%s'%target], df['D_%s'%target], df_trigger['%s_trigger'%target],
                        df_trigger['D_%s_trigger'%target],
                       pair_cut='and abs(h2_pid)==2212',minz=-1.5,maxz=3.0,nbins=20,variable='h2_rap')
                       
    plt.errorbar(np.subtract(x,-i*0.005),r,yerr=err,label='This work, %s'%target,ms=8,fmt='o',alpha=0.99,color=colors[target])
    plt.fill_between(np.subtract(x,-i*0.005), r*0.94,r*1.04, color=colors[target],alpha=0.2)
    
plt.xlabel('proton rapidity',fontsize=18)
plt.ylabel('$R_{2h}$',fontsize=18)

#plt.xlim([0.06,0.5])
#plt.ylim([0.0,2.0])

plt.axhline(y=1.0,color='black',linestyle='--')
plt.legend(loc='best',frameon=False,fontsize=15,ncol=1)


#plt.yscale('log')

plt.tight_layout()
plt.savefig('TheoryOnly.png')



In [ ]:
fig = plt.figure(figsize=(8,6))

for target in ['C','Fe','Pb']:
    r, err,x = getRatio(df['GiBUU_%s'%target],df['GiBUU_D'],
                        df_trigger['GiBUU_%s_trigger'%target],df_trigger['GiBUU_D_trigger'],
                        pair_cut='and abs(h2_pid)==2212 and h2_z>0.1 and h2_rap>-1.5 and h2_rap<-1.0',minz=0.0,maxz=3.14,nbins=20,variable='dphi')
    plt.errorbar(x,r,yerr=err,label='GiBUU %s'%target,ms=8,lw=2)
for target in ['C','Fe','Pb']:
    r, err,x = getRatio(df['%s'%target], df['D_%s'%target], df_trigger['%s_trigger'%target],
                        df_trigger['D_%s_trigger'%target],
                        pair_cut='and abs(h2_pid)==2212 and h2_z>0.1 and h2_rap>-1.5 and h2_rap<-1.0',minz=0.0,maxz=3.14,nbins=20,variable='dphi')
                       
    plt.errorbar(np.subtract(x,-i*0.005),r,yerr=err,label='This work, %s'%target,ms=8,fmt='o',alpha=0.99,color=colors[target])
    plt.fill_between(np.subtract(x,-i*0.005), r*0.94,r*1.04, color=colors[target],alpha=0.2)
plt.xlabel('$\Delta\phi$',fontsize=18)
plt.ylabel('$R_{2h}$',fontsize=18)

#plt.xlim([0.06,0.5])
#plt.ylim([0.0,2.0])

plt.axhline(y=1.0,color='black',linestyle='--')
plt.legend(loc='best',frameon=False,fontsize=15,ncol=1)


#plt.yscale('log')

plt.tight_layout()
plt.savefig('TheoryOnly.png')

In [ ]:
fig = plt.figure(figsize=(8,6))



for i,target in enumerate(['C','Fe','Pb']):
    #data
    r, err,x = getRatio(df['%s'%target], df['D_%s'%target], df_trigger['%s_trigger'%target], df_trigger['D_%s_trigger'%target]
                       ,variable='h2_z',minz=0.05,maxz=0.5
                       )
    plt.errorbar(np.subtract(x,-i*0.005),r,yerr=err,label='This work, %s'%target,ms=8,fmt='o',alpha=0.99,color=colors[target])
    plt.fill_between(np.subtract(x,-i*0.005), r*0.94,r*1.04, color=colors[target],alpha=0.2)
    
    
    #gibuu
    r, err,x = getRatio(df['GiBUU_%s'%target],df['GiBUU_D'],df_trigger['GiBUU_%s_trigger'%target],df_trigger['GiBUU_D_trigger']
                       ,variable='h2_z',minz=0.05,maxz=0.5, applyweight=False, pair_cut = ' and abs(h2_pid)==211'
                       )
    #plt.fill_between(x,r,r-err,r+err,label='GiBUU %s'%target,alpha=0.9,lw=5)
    plt.plot(x,r,label='GiBUU %s'%target,alpha=0.79,lw=3,color=colors[target])

#plt.xscale('log')
plt.xlabel('$z_2$',fontsize=22)
plt.ylabel('$R_{2h}$',fontsize=22)
plt.axhline(y=1.0,color='black',linestyle='--')
#plt.ylim([0.0,1.5])
plt.tight_layout()
plt.savefig('MainResult_withGiBUU_z2.png')


#plt.errorbar(hermes_x['N'], hermes_y['N'],yerr=hermes_ytotal['N'],fmt='o',label='Hermes, N',fillstyle=None,ms=8,capsize=5,color='grey')
#plt.errorbar(hermes_x['Kr'], hermes_y['Kr'],yerr=hermes_ytotal['Kr'],fmt='o',label='Hermes, Kr',fillstyle=None,ms=8,capsize=5,color='brown')
#plt.errorbar(hermes_x['Xe'], hermes_y['Xe'],yerr=hermes_ytotal['Xe'],fmt='o',label='Hermes, Xe',fillstyle=None,ms=8,capsize=5,color='darkviolet')
#plt.errorbar(neutrino_x, neutrino_y,yerr=neutrino_yerror,fmt='s',mfc='white',label='Neutrino, $A_{eff}=21$',fillstyle=None,ms=8,capsize=5)

plt.tick_params(axis="x", labelsize=16)
plt.tick_params(axis="y", labelsize=16)
#plt.text(0.20, 1.2, 'CLAS \nINTERNAL', fontsize=30)

plt.legend(frameon=False,ncol=2,fontsize=15)
plt.tight_layout()
plt.savefig('MainResult_withGiBUU_z2_withHERMES.png')
plt.savefig('MainResult_withGiBUU_z2_withHERMES.pdf')

In [ ]:
fig = plt.figure(figsize=(8,6))

colors = {}
colors['C'] = '#1f77b4'
colors['Fe'] = '#ff7f0e'
colors['Pb']=  '#2ca02c'

for i,target in enumerate(['C','Fe','Pb']):
    #data
    
  
    r, err,x = getRatio(df['%s'%target], df['D_%s'%target], df_trigger['%s_trigger'%target], df_trigger['D_%s_trigger'%target]
                       ,variable='h2_z',minz=0.07,maxz=0.5,applyweight=False
                       )
    plt.errorbar(np.subtract(x,0*0.0025),r,yerr=err,label='This work, %s'%target,ms=8,fmt='o',alpha=0.8,color=colors[target])

   
plt.xlabel('$z_2$',fontsize=22)
plt.ylabel('$R_{2h}$',fontsize=22)
plt.axhline(y=1.0,color='black',linestyle='--')
#plt.ylim([0.0,1.5])
plt.tight_layout()
plt.savefig('MainResult_withGiBUU_z2.png')


plt.tick_params(axis="x", labelsize=16)
plt.tick_params(axis="y", labelsize=16)

plt.legend(frameon=False,ncol=3,fontsize=15)
plt.tight_layout()
plt.savefig('MainResult_withGiBUU_z2_withHERMES.png')
plt.savefig('MainResult_withGiBUU_z2_withHERMES.pdf')

# As a function of the Dipion mass (GeV)
the variable is called 'pair_mass'

In [ ]:
fig = plt.figure(figsize=(8,6))

colors = {}
colors['C'] = '#1f77b4'
colors['Fe'] = '#ff7f0e'
colors['Pb']=  '#2ca02c'

for i,target in enumerate(['C','Fe','Pb']):
    r, err,x = getRatio(df['%s'%target], df['D_%s'%target], df_trigger['%s_trigger'%target], df_trigger['D_%s_trigger'%target]
                       ,variable='pair_mass',minz=0.4,maxz=2.0,nbins=11,pair_cut='and h2_z>0.10',applyweight=False
                       )
    plt.errorbar(x+0.02*i,r,yerr=err,label='%s'%target,ms=8,fmt='o',alpha=0.8,color=colors[target])
    plt.fill_between(x+0.02*i,r*1.04,r*0.96, alpha=0.2,color=colors[target])
    
    
    r, err,x = getRatio(df['GiBUU_%s'%target],df['GiBUU_D'],df_trigger['GiBUU_%s_trigger'%target],df_trigger['GiBUU_D_trigger']
                       ,variable='pair_mass',minz=0.4,maxz=2.0,nbins=11,pair_cut='and h2_z>0.10',applyweight=False
                       )
    #plt.fill_between(x,r,r-err,r+err,label='GiBUU %s'%target,alpha=0.5)
    plt.plot(x+0.02*i,r,color=colors[target],lw=3,alpha=0.8,label='GiBUU %s'%target)
plt.tick_params(axis="x", labelsize=16)
plt.tick_params(axis="y", labelsize=16)
plt.legend(frameon=False,ncol=2,loc='upper left')
plt.xlabel('Dipion mass (GeV)',fontsize=22)
plt.ylabel('$R_{2h}$',fontsize=22)
plt.axhline(y=1.0,color='black',linestyle='--')

#plt.text(0.60, 1.1, 'CLAS \nINTERNAL', fontsize=30)

#plt.ylim([0.0,1.2])
plt.tight_layout()
plt.savefig('MainResult_withGiBUU_mass.png')
plt.savefig('MainResult_withGiBUU_mass.pdf')